In [1]:
import numpy as np
from ripser import ripser

In [22]:
def border_matrix(lv, le):
    """Computes the border matrix.
    
    Yes it's quadratic (len(lv) * len(le)) but who cares now.
    
    :param lv: list of vertices
    :type lv: numpy.ndarray
    :param le: list of edges
    :type le: numpy.ndarray
    
    :return: the border matrix
    :rtype: numpy.ndarray
    """
    columns = len(lv)
    rows = len(le)
    M = np.zeros((rows, columns))
    for c, vertex in enumerate(lv):
        for r, edge in enumerate(le):
            if vertex in edge:
                start, end = edge[0], edge[1]
                M[r,c] = 1 if vertex == start else -1
    return M

def basis_from_edges(E):
    """Returns a sequence of the edges from the matrix E.
    
    Given the edge matrix we compute the basis
    for the chains by taking all the edges from E
    and returning the pairs [i,j].
    
    :param E: the edges matrix.
    :type E: numpy.ndarray
    
    :return: sequence of edges in form [i,j]
    :rtype: numpy.ndarray
    """
    ones = np.triu(E)
    return np.transpose(ones.nonzero())

import enum
class Operator(enum.Enum):
    BORDER = 0
    CO_BORDER = 1

def from_distance(D, operator=Operator.BORDER, threshold=np.infty):
    """Returns the operator matrix associated to the threshold filtration.
    
    :param D: The distance matrix for your data set.
    :type D: numpy.ndarry, shape = (k,k)
    
    :param operator: which operator matrix to compute.
    :type operator: Operator
    
    :param threshold: edges longer than threshold will not be computed.
    :type threshold: np.single | np.double | float
    
    :return: (matrix associated with (co)-border operator, edges matrix, edges base)
    
    :rtype: (numpy.ndarray, numpy.ndarray, numpy.ndarray)
    
    :important: the return matrix is expressed in the edge base
    """
    # edges[r,c] == True if there is an edge from r to c
    # that fits the threshold distance
    under_threshold = (D <= threshold) & (D > 0)
    edges = under_threshold.astype(int)
    # there is an edge between r and c when edges[r,c] == 1
    # but we need to put the correct sign in the (co)border
    # matrix as it is 1 IFF r < c and -1 otherwise
    ones = np.triu(edges)
    edge_base = basis_from_edges(ones)
    oriented_edges = ones - ones.T
    # now we have edges with orientation and we can compute
    # our border matrix in the basis computed
    (vertices_num, _) = D.shape
    matrix = border_matrix(range(vertices_num), edge_base)
    return matrix if operator is Operator.BORDER else matrix.T, oriented_edges, edge_base

In [23]:
# let's make the distance for a triangle
distance = np.array([
    [0, 3, 4],
    [3, 0, 6],
    [4, 5, 0]
])

op, edges, basis = from_distance(distance, operator=Operator.CO_BORDER)


In [24]:
def transform_basis(alpha, basis):
    """Transform the co-cycle alpha to a vector
    represented in basis.
    
    :param alpha: rows of k columns, up to k-1 are the vertices of
    my simplex, column k is the coefficent
    :type alpha: numpy.ndarray
    
    :param basis: rows containing the simplicis in my basis
    :type basis: numpy.ndarray
    
    :return: alpha represented in basis
    :rtype: numpy.ndarray
    """
    components = np.zeros((1, len(basis)), dtype=int)
    for a in alpha:
        for i, b in enumerate(basis):
            if np.all(a[:2] == b):
                components[0, i] = a[2]
                break
        else:
            # this will be executed if
            # break is never reached
            raise ValueError("simplex {} is not in basis".format(a[:-1]))
    return components

def circular_coordinates(p):
    """Transform to circular coordinates.
    
    Returns the same coordinates but modulo Z
    and in the range [0,1)
    
    :param p: coordinates
    :type p: numpy.ndarray
    :rtype: numpy.ndarray
    """
    fractional = p - np.trunc(p)
    correction = np.floor(fractional)
    return fractional - correction

In [25]:
a = np.array([
    [0, 1, 16],
    [0, 2, 1]
])
b = np.array([
    [0, 1, 16],
    [0, 2, 1]
])
basis = np.array([
    [0,2],
    [0,1],
    [1,2],
])
# outputs array([1, 1, 0])
transform_basis(a, basis)
# outputs array([1, 16, 0])
transform_basis(b, basis)

error = np.array([
    [1, 0, 1] # the simplex [0,1] is not in the basis
])
try:
    # this will error
    transform_basis(error, basis)
except:
    ...

In [26]:
from numpy.linalg import pinv as pseudoinverse
# compute the example matrix from the assignment
m = np.array([
    [-1, 1, 0],
    [-1, 0, 1],
    [0, -1, 1]
])
p = pseudoinverse(m)
circular_coordinates(p @ (- np.array([1, 0,0])))

array([3.33333333e-01, 6.66666667e-01, 5.48387791e-17])

In [27]:
def close_enough(vertices, distance, threshold=np.infty):
    """Compute if all vertex pairs are close enought.
    
    :param vertices: vertices in our n-cycle
    :type vertices: sequence
    
    :param distance: distance(i,j) return the distance between i and j
    :type distance: callable
    
    :param threshold: threshold distance
    :type threshold: float
    
    :rtype: bool
    """
    for i, vertex in enumerate(vertices):
        for other in vertices[i+1:]:
            if distance(vertex, other) > threshold:
                return False
    return True

def project(alpha, distance, threshold=np.infty):
    """Compute the representative (co)-cycle in the threshold filtration.
    
    :param alpha: the (co)-cycle to project
    :type alpha: numpy.ndarray
    
    :param distance: distance(i,j) return the distance between i and j.
    :type distance: callable
    
    :param threshold: exclude simplices over threshold
    :type threshold: float
    
    :return: alpha without the simplices over the threshold
    :rtype: numpy.ndarray
    """
    (r, _) = alpha.shape
    mask = np.full(r, False, dtype=bool)
    for i, simplex in enumerate(alpha):
        if close_enough(simplex[:-1], distance, threshold):
            mask[i] = True
    if not any(mask):
        raise ValueError("The feature does not exist for time: {}".format(threshold))
    return alpha[mask]

def compute(D, alpha, r, prime):
    """
    :param D: Distance matrix
    :param alpha: representative co-cycle
    :param r: threshold
    :param prime: prime field coefficients
    """
    def distance(i, j):
        """wraps distance matrix to make it callable"""
        return D[i,j]
    alpha_r = project(alpha, distance, r)
    lift = np.copy(alpha_r)
    cap = (prime-1)/2
    to_reduce = (alpha_r[:,-1] > cap).astype(int) # place a 1 when bigger than cap
    lift[:,-1] = alpha_r[:,-1] - prime * to_reduce
    co_border, edges, basis = from_distance(D, operator=Operator.CO_BORDER)
    b = transform_basis(lift, basis)
    p = pseudoinverse(co_border)
    y = p @ (- b.T)
    return co_border, edges, basis, b, y

In [28]:
_, _, _, _, y = compute(distance, a, np.infty, 17)
y, circular_coordinates(y)

(array([[ 0.66666667],
        [ 0.33333333],
        [-0.33333333]]),
 array([[0.66666667],
        [0.33333333],
        [0.66666667]]))

In [9]:
N=100
t = np.linspace(0, 2*3.1415,N , endpoint=False)
x = np.transpose([np.cos(t), np.sin(t)]) + 0.2*np.random.random((N,2)) # 

In [ ]:
result = ripser()